<a href="https://colab.research.google.com/github/jisusP/Data_science/blob/main/APA_a_Mendeley.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tipos de referencias en BibTex
**article**: Un artículo de un periódico o revista.

**book**: Un libro con una editorial que se indica en forma explícita. Los campos requeridos en este caso son author (autor), editor, title (título), publisher (editorial) y year (año).

**booklet**: Una obra que está impresa y encuadernada, pero sin una editorial o institución patrocinadora.

**conference**: Lo mismo que inproceedings, incluido para compatibilidad con el lenguaje de marcación Scribe.

**inbook**: Una parte de un libro, que puede ser un capítulo (o sección) o un rango de páginas.

**incollection**: Una parte de un libro que tiene su propio título.

**inproceedings**: Un artículo en las actas de sesiones (proceedings) de una conferencia.

**manual**: Documentación técnica.

**mastersthesis**: Una tesis de maestría (Master thesis) o proyecto fin de carrera.

**misc**: Para uso cuando los demás tipos no corresponden.

**phdthesis**: Una tesis de doctorado (Ph.D. thesis).

**proceedings**: Las actas de sesiones de una conferencia.

**techreport**: Un reporte publicado por una escuela u otra institución, usualmente numerado dentro de una serie.

**unpublished**: Un documento que tiene un autor y título, pero que no fue formalmente publicado.


## *Formato de cita*
---
@incollection{SINGH201865,

title = {Chapter 5 - Target Product Profile and Clinical Development Plan},

editor = {Divya Vohora and Gursharan Singh},

booktitle = {Pharmaceutical Medicine and Translational Clinical Research},

publisher = {Academic Press},

address = {Boston},

pages = {65-80},

year = {2018},

isbn = {978-0-12-802103-3},

doi = {https://doi.org/10.1016/B978-0-12-802103-3.00005-5},

url = {https://www.sciencedirect.com/science/article/pii/B9780128021033000055},

author = {Gursharan Singh},

keywords = {Target Product Profile, Clinical Development Plan, TPP Process, Regulatory, Commercial, Quality TPP, Phases of Development, Study Variables, Bias, Control Group, Study Design, Sample Size},

abstract = {The ultimate aim of a drug development program is to obtain regulatory approval with an optimal label (prescribing information) that which ensures commercial success. Target Product Profile (TPP) is the intended profile of a pharmaceutical/biotechnology product or technology, developed based on cross-functional discussions. The regulatory TPP is organized according to key sections in a drug label, while an abbreviated version of the TPP is used for commercial/business communications including net product value calculation. Once the TPP is finalized, the clinical development is planned to meet the criteria specified in the TPP. The Clinical Development Plan is the blueprint of the entire clinical research strategy of a drug which defines the critical path for the clinical program including development assessment and decision points and the project resource (personnel and budget) estimates. This chapter covers the basic concepts of clinical development including phases of development, study endpoints, design configurations, control groups, etc., as well as the content of the Clinical Development Plan.}
}


In [107]:
import re
import glob
from natsort import natsorted

def procesar_referencias(archivo_entrada):
    with open(archivo_entrada, 'r', encoding='utf-8') as archivo:
      #Expresión regular, seguido de un corchete, un d+ (es 1 o más dígitos) y cierra con un corchete
        referencias = re.split(r'\[\d+\]', archivo.read())
        ref_sin_espacios = list(filter(lambda x: x.strip(), referencias))
        ref_sin_saltosLinea = [i.replace("\n", "") for i in ref_sin_espacios] #Me elimina los elementos que no tienen nada
        print(ref_sin_saltosLinea)

        REF={n:{info} for n, info in enumerate(ref_sin_saltosLinea)}
        print(REF)
        print(ref_sin_saltosLinea[0])
        l_doi = []
        l_year = []
        l_isbn = []
        l_url = []
        l_title = []
        l_author = []
        l_publisher = []
        l_fullname = []
        pp=[]
        tipo_ref = {}
        id = {}
        c = 0
        for ref in ref_sin_saltosLinea:

            patron_title = r"(?<=\)\.)\s*([^.]+)"
            resultado_title = re.search(patron_title, ref)
            title = resultado_title.group() if resultado_title else ""

            """if title == " (2018)":
                print("###################")
                print(ref)"""

            patron_isbn = r"(ISBN|isbn)([\w\.-/]+)"
            resultado_isbn = re.search(patron_isbn, ref)
            isbn = resultado_isbn.group() if resultado_isbn else ""

            patron_year = r"(?#[\s?\(])(\d{4,})(?#[\s?\)-?.?,?])"
            resultado_year = re.search(patron_year, ref)
            year = resultado_year.group() if resultado_year else ""

            patron_doi = r"(https?://doi\.org/|doi:)([\w\.-/]+)"
            resultado_doi = re.search(patron_doi, ref)
            doi = resultado_doi.group() if resultado_doi else ""

            patron_url = r"(https://|http://)(?!doi|dx)([\w\W]+)"
            resultado_url = re.search(patron_url, ref)
            url = resultado_url.group() if resultado_url else ""

            patron_author = r"([^\(]+) (?=\(\d{4})"
            resultado_author = re.search(patron_author, ref)
            author = resultado_author.group() if resultado_author else ""


            #patron_pub = r"(?#[.?])(?<=\s)[^.,:]+(?!\.\s*https)" #(?=(https|http|\d|\w|$)?)(?#[.,:]\s*)"
            patron_pub = r"([^.,:]+)(?=\s?[,.:](?#doi|\d|\w|$))"#r"([^.,:]+)(?=[.,:]\s?https?|http?|\d?|\w?|$)"#(?<=\.|\?\s)r"(?#[.?]\s*)([^.:,]+)(?#[.:,]\s*)"
            title2 = re.escape(title) # Escapar los caracteres especiales en el título
            res_parcial_pub = re.search(title2, ref)
            ##########################


            if res_parcial_pub and resultado_doi:
                resultado_pub = re.search(patron_pub, ref[res_parcial_pub.end():resultado_doi.start()])
                pp.append(ref[res_parcial_pub.end():resultado_doi.start()])
                pp.append(0)
                publisher = resultado_pub.group() if resultado_pub else ""
                l_publisher.append(publisher)
            elif res_parcial_pub:
                a = ref[res_parcial_pub.end():].strip()

                if a.startswith("."):
                  a = a[1:].strip()
                if a == url:
                    l_publisher.append("")
                    pp.append("X")
                elif title.strip() in a:
                    l_publisher.append("")
                    pp.append("Y")
                else:
                    resultado_pub = re.search(patron_pub, ref[res_parcial_pub.end():])
                    pp.append(ref[res_parcial_pub.end():])
                    pp.append(1)
                    publisher = resultado_pub.group() if resultado_pub else ""
                    l_publisher.append(publisher)

            if url in ref and url:
                tipo_ref[c] = "web"
            elif doi in ref and (isbn in ref or 'chapter' in ref or 'PMID' in ref or 'ISSN' in ref) and doi and isbn:
                tipo_ref[c] = "incollection"
            elif isbn in ref and isbn:
                tipo_ref[c] = "book"
            else:
                tipo_ref[c] = "article"

            name_match = re.search(r"(\w+[^\.])", author)
            name = name_match.group() if name_match else ""
            #name = re.search(r"(\w+[^\.])", author).group()
            name_nitido = name.replace('.', '').replace(',','') if name else ""
            id[c] = f"{name_nitido.strip()}{year}"

            l_doi.append(doi)
            l_year.append(year)
            l_isbn.append(isbn)
            l_url.append(url)
            l_title.append(title)
            l_author.append(author)


            with open(f"{c}{id[c]}.bib", "w") as plantilla:

                plantilla.write(f"@{tipo_ref[c]}{{{id[c]},\n")
                if title:
                    plantilla.write(f"title = {{{title.strip()}}},\n")
                if l_publisher[c]:
                    if tipo_ref[c] == "book" or tipo_ref[c] == "web":
                        plantilla.write(f"publisher = {{{publisher.strip()}}},\n")
                    else:
                        plantilla.write(f"journal = {{{publisher.strip()}}},\n")
                if year:
                    plantilla.write(f"year = {{{year}}},\n")
                if isbn:
                    plantilla.write(f"isbn = {{{isbn}}},\n")
                if doi:
                    plantilla.write(f"doi = {{{doi}}},\n")
                if url:
                    plantilla.write(f"url = {{{url}}},\n")
                if author:
                    aut = author.strip()
                    cadena = aut.replace('.', '')
                    nombres_apellidos = cadena.split(',')

                    if len(nombres_apellidos) > 1:
                        nombres_apellidos = [nombre_apellido.strip().split(' ') for nombre_apellido in nombres_apellidos]

                        first=[]
                        last =[]
                        for i,j in enumerate(nombres_apellidos):

                            if "&" not in j:
                                if i%2 == 0:
                                    first.append(j[0])
                                else:
                                    last.append(j[0])
                            else:
                                if i%2 == 0:
                                    first.append(j[1])
                                else:
                                    last.append(j[0])
                        full = []
                        for i in range(len(last)):
                            full.append(f"{last[i]}., {first[i]}")
                        full_name = [" and ".join(full)]
                        l_fullname.append(full_name)
                        plantilla.write(f"author = {{{full_name[0]}}}}}")
                    else:
                        # Caso con nombre de la empresa o entidad
                        full_name = [aut]
                        plantilla.write(f"author = {{{{{full_name[0]}}}}}}}")


            c +=1

        print(l_doi)
        print(l_year)
        print(l_isbn)
        print(l_url)
        print(l_title)
        print(l_author)
        print(l_publisher)
        print(l_fullname)
        print("\n")

        """if (len(l_doi) == len(l_year) == len(l_isbn) == len(l_url) == len(l_title) == len(l_author)==len(l_publisher)):
          print("No faltó ninguna referencia por fuera")
        else:
          print("Hay alguna que se escapa, look at it!")
          print(len(l_doi), len(l_title))
          print(len(l_publisher))
        print("\n")
        repeti, normal = {}, {}
        for i, http in enumerate(l_publisher):

          if http == " http" or http =="" or http == " https" or http == "https":
            repeti[i] = http
          normal[i] =http
        print(repeti)
        print("\n")
        print(normal)
        #print(l_publisher[27])
        #print(l_title[27])
        print("\n", pp)"""

        print(l_doi[92])
        print(l_year[92])
        print(l_isbn[92])
        print(l_url[92])
        print(l_title[92])
        print(l_author[92])
        print(l_publisher[92])



procesar_referencias("ref_complete.txt")
#procesar_referencias("ref1.txt")

import os
from zipfile import ZipFile
from google.colab import files

# Ruta de la carpeta que contiene los archivos .bib
carpeta = "/content/"
# Crear un archivo zip y agregar los archivos .bib
with ZipFile("/content/archivos.zip", "w") as zip:
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".bib"):
            ruta_completa = os.path.join(carpeta, archivo)
            zip.write(ruta_completa, os.path.basename(ruta_completa))
# Descargar el archivo zip
files.download("/content/archivos.zip")
# Borrar archivos .bib
# Borrar archivos .bib
for archivo in os.listdir(carpeta):
    if archivo.endswith(".bib"):
        ruta_completa = os.path.join(carpeta, archivo)
        os.remove(ruta_completa)

[' Saldívar-González, F., Prieto-Martínez, F. D., & Medina-Franco, J. L. (2016). Descubrimiento y desarrollo de fármacos: un enfoque computacional. Educación Química. https://doi.org/10.1016/j.eq.2016.06.002', ' Lynch, S. S. (2022). Efectividad y seguridad de un fármaco. MSD Manuals. University of California San Francisco School of Pharmacy. https://www.msdmanuals.com/es/hogar/f%C3%A1rmacos-o-sustancias/introducci%C3%B3n-a-los-f%C3%A1rmacos/efectividad-y-seguridad-de-un-f%C3%A1rmaco', ' U.S. Food and Drug Administration. (2021, August 31). Development & Approval Process (Drugs). https://www.fda.gov/drugs/development-approval-process-drugs', ' Hughes, J. P., Rees, S., Kalindjian, S. B., Philpott, K. L. (2011). Principles of early drug discovery. British Journal of Pharmacology, 162(6), 1239-1249. doi:10.1111/j.1476-5381.2010.01127.x', ' DiMasi, J. A., Grabowski, H. G., & Hansen, R. W. (2016). Innovation in the pharmaceutical industry: New estimates of R&D costs. Journal of Health Econom

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [82]:
import os
from zipfile import ZipFile
from google.colab import files

# Ruta de la carpeta que contiene los archivos .bib
carpeta = "/content/"
# Crear un archivo zip y agregar los archivos .bib
with ZipFile("/content/archivos.zip", "w") as zip:
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".bib"):
            ruta_completa = os.path.join(carpeta, archivo)
            zip.write(ruta_completa, os.path.basename(ruta_completa))
# Descargar el archivo zip
files.download("/content/archivos.zip")
# Borrar archivos .bib
# Borrar archivos .bib
for archivo in os.listdir(carpeta):
    if archivo.endswith(".bib"):
        ruta_completa = os.path.join(carpeta, archivo)
        os.remove(ruta_completa)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(l_doi[92])
print(l_year[92])
print(l_isbn[92])
print(l_url[92])
print(l_title[92])
print(l_author[92])
print(l_publisher[92])
print(l_fullname[92])